### MySQL 접속용 샘플 프로그램
- 접속 정보가 있는 파일 읽기

In [2]:
# pip install pymysql
import json
with open('./mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'ysuser',
 'password': 'yspass',
 'database': 'world',
 'port': 3306}

- MySQL 접속

In [24]:
import pymysql
conn = pymysql.connect(
    host=config['host'], user=config['user'], password=config['password'], 
    database=config['database'], port=config['port']
)

In [5]:
# city table에서 10개 데이터 읽기
cur = conn.cursor()
sql ="SELECT * FROM city LIMIT 10;"
cur.execute(sql)
rows = cur.fetchall()
cur.close()

for row in rows:
    print(row)

(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)
(6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321)
(7, 'Haag', 'NLD', 'Zuid-Holland', 440900)
(8, 'Utrecht', 'NLD', 'Utrecht', 234323)
(9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843)
(10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)


In [25]:
# 레코드 하나만 
cur = conn.cursor()
sql ="SELECT * FROM city WHERE name=%s;"
cur.execute(sql,('수원',))
row = cur.fetchone()
cur.close()
print(row)

(2340, '수원', 'KOR', 'Kyonggi', 1200000)


In [6]:
import pandas as pd
df = pd.DataFrame(rows,columns=['id','name','countrycode','district','population'])
df.set_index('id',inplace=True)
df

,name,countrycode,district,population
id,,,,
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200
6,Rotterdam,NLD,Zuid-Holland,593321
7,Haag,NLD,Zuid-Holland,440900
8,Utrecht,NLD,Utrecht,234323
9,Eindhoven,NLD,Noord-Brabant,201843


- 검색: 국가코드 KOR 의 인구수 Top 10

In [13]:
cur = conn.cursor()
sql ="SELECT * FROM city WHERE countrycode=%s ORDER BY population DESC LIMIT %s;"
cur.execute(sql, ('KOR', 10))       # 파라메타는 반드시 튜플로 전달해야함
rows = cur.fetchall()
cur.close()

for row in rows:
    print(row)

(2331, 'Seoul', 'KOR', 'Seoul', 9981619)
(4085, '대도시', 'KOR', 'Kyonggi', 8000000)
(2332, 'Pusan', 'KOR', 'Pusan', 3804522)
(2333, 'Inchon', 'KOR', 'Inchon', 2559424)
(2334, 'Taegu', 'KOR', 'Taegu', 2548568)
(2335, 'Taejon', 'KOR', 'Taejon', 1425835)
(2336, 'Kwangju', 'KOR', 'Kwangju', 1368341)
(2340, '수원', 'KOR', 'Kyonggi', 1200000)
(2337, 'Ulsan', 'KOR', 'Kyongsangnam', 1084891)
(2338, 'Songnam', 'KOR', 'Kyonggi', 869094)


- 데이터 추가(Insert)

In [14]:
cur = conn.cursor()
sql ="INSERT INTO city VALUES(DEFAULT, 'Java','KOR','Kyonggi',6000000);"
cur.execute(sql)
conn.commit()       # DB가 변화되는SQL(INSERT,DELETE,UPDATE)에서는 필수
cur.close()

In [16]:
cur = conn.cursor()
params = ('Node.js','KOR','Kyonggi',6000000)
sql ="INSERT INTO city VALUES(DEFAULT, %s, %s, %s,%s);"
cur.execute(sql,params)
conn.commit()       # DB가 변화되는SQL(INSERT,DELETE,UPDATE)에서는 필수
cur.close()

In [17]:
params_list = [
    ('HTML','KOR','Kyonggi',7000000),
    ('CSS','KOR','Kyonggi',5000000)
]
cur = conn.cursor()
sql ="INSERT INTO city VALUES(DEFAULT, %s, %s, %s,%s);"
for params in params_list:
    cur.execute(sql,params)
# cur.execute(sql,params_list) 대량 처리는 어려움
conn.commit()       # DB가 변화되는SQL(INSERT,DELETE,UPDATE)에서는 필수
cur.close()

- 수정(UPDATE)

In [18]:
cur = conn.cursor()
sql ="UPDATE city SET `Name`='CSS' WHERE id=4090;"
cur.execute(sql)
conn.commit()       
cur.close()

In [19]:
params_list=[('HTML5',4089),('CSS3',4090)]
cur = conn.cursor()
sql ="UPDATE city SET `Name`=%s WHERE id=%s;"
cur.executemany(sql,params_list)
conn.commit()       
cur.close()

In [21]:
# delete
cur = conn.cursor()
sql ="DELETE FROM city WHERE id>=%s;"
cur.execute(sql,(4085,))
conn.commit()       
cur.close()

- MySQL 접속 해제

In [26]:
conn.close()